In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 스타일 설정
sns.set(style="whitegrid")

In [13]:
# 데이터 파일 경로 설정 (필요 시 수정)
file_path = 'nbasal.xlsx'

# 엑셀 파일 불러오기
df = pd.read_excel(file_path, engine='openpyxl')

# 데이터의 처음 몇 행 확인
print(df.head())

# 데이터 요약 통계 확인
print(df.describe())

# 결측치 확인
print(df.isnull().sum())


     wage  exper  age  educ  games  points  rebounds  assists     lwage  black
0  1.0025      4   27     4   77.0    15.5       3.9      4.5  2.305082      1
1  2.0300      5   28     4   78.0    13.3       2.5      8.8  3.010621      1
2  0.6500      1   25     4   74.0     5.5       3.3      0.2  1.871802      1
3  1.0500      5   29     4   82.0    11.6       4.0      NaN  2.351375      1
4  2.0300      5   28     4   47.0     7.3       5.1      1.5  3.010621      1
             wage       exper         age        educ       games      points  \
count  278.000000  291.000000  291.000000  291.000000  289.000000  286.000000   
mean     1.421261    5.254296   27.587629    3.704467   65.072664    9.974825   
std      1.001932    3.529766    3.549898    0.797808   19.593568    5.868620   
min      0.150000    1.000000   21.000000    0.000000    0.000000    0.000000   
25%      0.650000    2.000000   25.000000    4.000000   55.000000    5.225000   
50%      1.173000    5.000000   27.00000

In [19]:
# 결측치 확인
missing_values = df.isnull().sum()
print("\n결측치 개수:")
print(missing_values)
# 결측치 값이 있는 행 제거
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# 제거 후 데이터 확인
print("\n결측치 제거한 후 데이터 개수:")
print(df_clean.shape)



결측치 개수:
wage        13
exper        0
age          0
educ         0
games        2
points       5
rebounds     5
assists      3
lwage       13
black        0
dtype: int64

결측치 제거한 후 데이터 개수:
(271, 10)


1. 요약통계량 보고

In [20]:
# 요약 통계량 계산
summary_stats = df[['wage', 'lwage']].describe()

print("Summary Statistics for 'wage' and 'lwage':")
print(summary_stats)


Summary Statistics for 'wage' and 'lwage':
             wage       lwage
count  271.000000  271.000000
mean     1.430663    2.352965
std      0.999976    0.880881
min      0.150000    0.405465
25%      0.650000    1.871802
50%      1.195000    2.480731
75%      2.022250    3.006788
max      5.740000    4.050045


2. 회귀분석 및 해석

In [21]:
# 회귀 모형 추정
independent_vars = ['age', 'exper', 'games', 'points', 'rebounds', 'assists', 'black']

X = df[independent_vars]
y = df['lwage']

# 상수항 추가
X = sm.add_constant(X)

# OLS 회귀 모형 적합
model = sm.OLS(y, X).fit()

# 회귀 결과 요약 출력
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     35.82
Date:                Sat, 12 Oct 2024   Prob (F-statistic):           6.05e-35
Time:                        21:23:28   Log-Likelihood:                -258.94
No. Observations:                 271   AIC:                             533.9
Df Residuals:                     263   BIC:                             562.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9680      0.830      2.372      0.0

games변수의 계수는 0.0012로 games가 한 단위 증가할 때 wage는 약 1.2%증가한다고 해석 가능

  2. black계수는 0.0534로 더미 변수인데 계수가 0보다 크기 때문에 흑인 선수의 임금이 비흑인 선수의 임금보다 크다는 것을 알 수 있다

3. age의 경우 p-value가 0.05보다 크기 때문에 귀무가설을 기각하지 못한다. 즉 유의미한 영향을 미치지 못함을 알 수 있다.

In [22]:
p_value_age = model.pvalues['age']
print(f"P-value for age: {p_value_age}")

# 유의수준 설정
alpha = 0.05

if p_value_age < alpha:
    print("p-value < 0.05 → 귀무가설을 기각합니다.")
    print("age가 log(wage)에 유의미한 영향을 미칩니다.")
else:
    print("p-value ≥ 0.05 → 귀무가설을 기각하지 못합니다.")
    print("age가 log(wage)에 유의미한 영향을 미치지 않습니다.")

P-value for age: 0.20011802448392518
p-value ≥ 0.05 → 귀무가설을 기각하지 못합니다.
age가 log(wage)에 유의미한 영향을 미치지 않습니다.


In [23]:
# 제한된 독립 변수 리스트 (age와 exper 제외)
restricted_vars = ['games', 'points', 'rebounds', 'assists', 'black']

# 제한된 독립 변수와 종속 변수 설정
X_restricted = df[restricted_vars]
y_restricted = df['lwage']

# 상수항 추가 (절편)
X_restricted = sm.add_constant(X_restricted)

# 제한된 회귀 모델 적합
model_restricted = sm.OLS(y_restricted, X_restricted).fit()

# 제한된 회귀 결과 요약 출력
print(model_restricted.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     35.73
Date:                Sat, 12 Oct 2024   Prob (F-statistic):           6.76e-28
Time:                        21:29:12   Log-Likelihood:                -279.84
No. Observations:                 271   AIC:                             571.7
Df Residuals:                     265   BIC:                             593.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1946      0.164      7.296      0.0

In [24]:
# F-검정 수행
f_test = model.compare_f_test(model_restricted)
f_stat = f_test[0]
p_value = f_test[1]
df_diff = f_test[2]

print(f"F-statistic: {f_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom Difference: {df_diff}")

# 유의수준 설정
alpha = 0.05

if p_value < alpha:
    print("\nP-value < 0.05 → 귀무가설을 기각합니다.")
    print("age와 exper는 log(wage)에 유의미한 영향을 미칩니다.")
else:
    print("\nP-value ≥ 0.05 → 귀무가설을 기각하지 못합니다.")
    print("age와 exper는 log(wage)에 유의미한 영향을 미치지 않습니다.")


F-statistic: 21.936876151798575
P-value: 1.5451641716468364e-09
Degrees of Freedom Difference: 2.0

P-value < 0.05 → 귀무가설을 기각합니다.
age와 exper는 log(wage)에 유의미한 영향을 미칩니다.


age의 t검정 결과로는 유의미한 영향을 미치지 않는다고 나왔으나 f검정 결과로는 유의미한 영향을 미쳤다고 나왔음.  
때문에 다중공선성이 있을 확률이 높음

In [26]:
# Experience의 회귀 계수와 p-value 확인
beta_exper = model.params['exper']
se_exper = model.bse['exper']
t_stat_exper = beta_exper / se_exper
p_value_exper = 2 * (1 - stats.t.cdf(abs(t_stat_exper), df=model.df_resid))
print(f"\n### Experience의 회귀 계수 = {beta_exper}")
print(f"Experience의 표준 오차 = {se_exper}")
print(f"Experience의 t-통계량 = {t_stat_exper}")
print(f"Experience의 p-값 = {p_value_exper}")



### Experience의 회귀 계수 = 0.11951926913482087
Experience의 표준 오차 = 0.03566600702023304
Experience의 t-통계량 = 3.351069523062073
Experience의 p-값 = 0.0009231324752814185


In [27]:
# Age와 Experience 간의 상관관계
correlation = df_clean['age'].corr(df_clean['exper'])
print(f"\n### Age와 Experience 간의 상관계수 = {correlation}")



### Age와 Experience 간의 상관계수 = 0.9410094370990791


만약 다중공선성이 여전히 문제가 될 경우 ridge혹은 lasso의 방법을 적용가능